I wanted to rename some of the columns so they would make more sense.

I then needed to aggregate the data by invoice number and sum the Credit columns and Debit columns.  After getting the total debit and credits for an invoice, need to determine the total invoice amount by subtracting the total credits from total debits.

To determine if an invoice for a customer was a repeat customer, I first sorted the data by customer and date.  A new column called 'Repeat Customer' was added to the dataset with an 'n' if this was a first-time invoice for a customer and a 'y' if this was a repeat customer, a customer that had bought from us before. 

In [29]:
import sqlite3 as sl
import pandas as pd
import numpy as np

In [30]:
# Read csv files

sales_df = pd.read_csv('Sales.csv')
cust_df = pd.read_csv('Customer.csv')

In [31]:
sales_df.head()

,Unnamed: 0,Customer ID,Account ID,Account Description,Invoice/CM #,Date,Debit Amnt,Credit Amnt,Last Inv Date,Sales Rep ID,Job ID
0,0,WATERO,30020,Sales - Turf Repair,37144,2020-04-01,0.0,2607.75,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE"
1,1,WATERO,30020,Sales - Turf Repair,37147,2020-04-01,0.0,3806.04,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE"
2,2,WATERO,30020,Sales - Turf Repair,37148,2020-04-02,0.0,2070.17,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE"
3,3,WATERO,30020,Sales - Turf Repair,37155,2020-04-03,0.0,3558.18,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE"
4,4,WATERO,30020,Sales - Turf Repair,37172,2020-04-10,0.0,2825.81,2021-12-29,JAMESV,"20001-02JV,15-Turf Repair,00-REVENUE"


In [32]:
cust_df.head()

,Unnamed: 0,Customer ID,Cust Since,City,State,Zip
0,0,103,2008-07-21,OVERLAND PARK,KS,66210
1,1,10975,2008-07-21,OVERLAND PARK,KS,66210
2,2,1509WALN,2018-12-12,KANSAS CITY,MO,64106
3,3,1617,2008-07-15,NaN,NaN,NaN
4,4,1906WYAN,2019-01-28,KANSAS CITY,MO,64108


In [33]:
sales_df.describe

<bound method NDFrame.describe of        Unnamed: 0 Customer ID  Account ID    Account Description Invoice/CM #  \
0               0      WATERO       30020    Sales - Turf Repair        37144   
1               1      WATERO       30020    Sales - Turf Repair        37147   
2               2      WATERO       30020    Sales - Turf Repair        37148   
3               3      WATERO       30020    Sales - Turf Repair        37155   
4               4      WATERO       30020    Sales - Turf Repair        37172   
...           ...         ...         ...                    ...          ...   
58219       58226        VALL       30900  Sales - Grounds Maint        39206   
58220       58227        VALL       30900  Sales - Grounds Maint        39206   
58221       58228        WOOD       30900  Sales - Grounds Maint        39241   
58222       58229        VALM       30900  Sales - Grounds Maint        39328   
58223       58230       ELITE       30900  Sales - Grounds Maint        393

In [34]:
# rename columns
sales_df = sales_df.rename(columns={'Customer ID':'Cust_ID','Date':'Inv_Date','Account ID':'Acct_ID',
                      'Invoice/CM #':'Inv_Num','Debit Amnt':'Debit_Amt', 'Credit Amnt':'Credit_Amt',
                      'Last Inv Date':'Last_Inv_Date','Account Description':'Acct_Desc',
                      'Sales Rep ID':'Sales_Rep_ID','Job ID':'Job_ID'
                    })
sales_df.head(5)

,Unnamed: 0,Cust_ID,Acct_ID,Acct_Desc,Inv_Num,Inv_Date,Debit_Amt,Credit_Amt,Last_Inv_Date,Sales_Rep_ID,Job_ID
0,0,WATERO,30020,Sales - Turf Repair,37144,2020-04-01,0.0,2607.75,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE"
1,1,WATERO,30020,Sales - Turf Repair,37147,2020-04-01,0.0,3806.04,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE"
2,2,WATERO,30020,Sales - Turf Repair,37148,2020-04-02,0.0,2070.17,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE"
3,3,WATERO,30020,Sales - Turf Repair,37155,2020-04-03,0.0,3558.18,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE"
4,4,WATERO,30020,Sales - Turf Repair,37172,2020-04-10,0.0,2825.81,2021-12-29,JAMESV,"20001-02JV,15-Turf Repair,00-REVENUE"


In [35]:
# rename columns for customer
cust_df = cust_df.rename(columns={'Customer':'Cust_ID','Cust Since':'Cust_Since_Dt'
                    })
cust_df.head(5)

,Unnamed: 0,Customer ID,Cust_Since_Dt,City,State,Zip
0,0,103,2008-07-21,OVERLAND PARK,KS,66210
1,1,10975,2008-07-21,OVERLAND PARK,KS,66210
2,2,1509WALN,2018-12-12,KANSAS CITY,MO,64106
3,3,1617,2008-07-15,NaN,NaN,NaN
4,4,1906WYAN,2019-01-28,KANSAS CITY,MO,64108


In [36]:
# Sum all the credit amounts for the same invoice number

sales_df['Inv_Total_Credit'] = sales_df.groupby('Inv_Num')['Credit_Amt'].transform('sum')

In [37]:
# Look at the Landscape sales, Acct ID 30200

print(sales_df.loc[sales_df['Acct_ID'] == 30200])


       Unnamed: 0 Cust_ID  Acct_ID            Acct_Desc     Inv_Num  \
918           918   MELOJ    30200  Sales - Landscaping        6428   
919           919   MELOJ    30200  Sales - Landscaping        6428   
920           920    NEAL    30200  Sales - Landscaping  02/03/05DS   
921           921   HINTC    30200  Sales - Landscaping        6435   
922           922   HINTC    30200  Sales - Landscaping        6435   
...           ...     ...      ...                  ...         ...   
17800       17806    KANG    30200  Sales - Landscaping       39243   
17801       17807    MILG    30200  Sales - Landscaping       39244   
17802       17808   PETER    30200  Sales - Landscaping       39245   
17803       17809   BICKM    30200  Sales - Landscaping       39253   
17804       17810    MILG    30200  Sales - Landscaping     39244CM   

         Inv_Date  Debit_Amt  Credit_Amt Last_Inv_Date Sales_Rep_ID  \
918    2005-01-19        0.0       180.0    2012-04-23          TOM   
919  

In [38]:
# Sum all the debits for each invoice

sales_df['Inv_Total_Debit'] = sales_df.groupby('Inv_Num')['Debit_Amt'].transform('sum')

In [39]:
# Look at the Landscape sales, Acct ID 30200
print(sales_df.loc[sales_df['Acct_ID'] == 30200])

       Unnamed: 0 Cust_ID  Acct_ID            Acct_Desc     Inv_Num  \
918           918   MELOJ    30200  Sales - Landscaping        6428   
919           919   MELOJ    30200  Sales - Landscaping        6428   
920           920    NEAL    30200  Sales - Landscaping  02/03/05DS   
921           921   HINTC    30200  Sales - Landscaping        6435   
922           922   HINTC    30200  Sales - Landscaping        6435   
...           ...     ...      ...                  ...         ...   
17800       17806    KANG    30200  Sales - Landscaping       39243   
17801       17807    MILG    30200  Sales - Landscaping       39244   
17802       17808   PETER    30200  Sales - Landscaping       39245   
17803       17809   BICKM    30200  Sales - Landscaping       39253   
17804       17810    MILG    30200  Sales - Landscaping     39244CM   

         Inv_Date  Debit_Amt  Credit_Amt Last_Inv_Date Sales_Rep_ID  \
918    2005-01-19        0.0       180.0    2012-04-23          TOM   
919  

In [40]:
# check number of rows in dataframe
sales_df.shape

(58224, 13)

In [41]:
# Drop the duplicate records for invoice number not that we have our total debit and total credit columns

sales_df = sales_df.drop_duplicates(subset=['Inv_Num'])

In [42]:
# Checking to make sure rows were dropped
sales_df.shape

(32816, 13)

In [43]:
# Look at my sales dataset to confirm new columns
sales_df.head(5)

,Unnamed: 0,Cust_ID,Acct_ID,Acct_Desc,Inv_Num,Inv_Date,Debit_Amt,Credit_Amt,Last_Inv_Date,Sales_Rep_ID,Job_ID,Inv_Total_Credit,Inv_Total_Debit
0,0,WATERO,30020,Sales - Turf Repair,37144,2020-04-01,0.0,2607.75,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE",2607.75,0.0
1,1,WATERO,30020,Sales - Turf Repair,37147,2020-04-01,0.0,3806.04,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE",3806.04,0.0
2,2,WATERO,30020,Sales - Turf Repair,37148,2020-04-02,0.0,2070.17,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE",2070.17,0.0
3,3,WATERO,30020,Sales - Turf Repair,37155,2020-04-03,0.0,3558.18,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE",3558.18,0.0
4,4,WATERO,30020,Sales - Turf Repair,37172,2020-04-10,0.0,2825.81,2021-12-29,JAMESV,"20001-02JV,15-Turf Repair,00-REVENUE",2825.81,0.0


In [44]:
# Calculate the total sale for an invoice by subtracting the debit from credit

sales_df.loc[:,'Inv_Total_Sale'] = sales_df['Inv_Total_Credit'] - sales_df['Inv_Total_Debit']
sales_df.head()

,Unnamed: 0,Cust_ID,Acct_ID,Acct_Desc,Inv_Num,Inv_Date,Debit_Amt,Credit_Amt,Last_Inv_Date,Sales_Rep_ID,Job_ID,Inv_Total_Credit,Inv_Total_Debit,Inv_Total_Sale
0,0,WATERO,30020,Sales - Turf Repair,37144,2020-04-01,0.0,2607.75,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE",2607.75,0.0,2607.75
1,1,WATERO,30020,Sales - Turf Repair,37147,2020-04-01,0.0,3806.04,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE",3806.04,0.0,3806.04
2,2,WATERO,30020,Sales - Turf Repair,37148,2020-04-02,0.0,2070.17,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE",2070.17,0.0,2070.17
3,3,WATERO,30020,Sales - Turf Repair,37155,2020-04-03,0.0,3558.18,2021-12-29,JAMESV,"20001-02JV,02-PLANTING,00-REVENUE",3558.18,0.0,3558.18
4,4,WATERO,30020,Sales - Turf Repair,37172,2020-04-10,0.0,2825.81,2021-12-29,JAMESV,"20001-02JV,15-Turf Repair,00-REVENUE",2825.81,0.0,2825.81


In [45]:
# Check number of columns
sales_df.shape

(32816, 14)

In [46]:
print(sales_df.loc[sales_df['Acct_ID'] == 30200])

       Unnamed: 0  Cust_ID  Acct_ID            Acct_Desc     Inv_Num  \
918           918    MELOJ    30200  Sales - Landscaping        6428   
920           920     NEAL    30200  Sales - Landscaping  02/03/05DS   
921           921    HINTC    30200  Sales - Landscaping        6435   
923           923  HORIZON    30200  Sales - Landscaping        6441   
925           925     PEPN    30200  Sales - Landscaping        6442   
...           ...      ...      ...                  ...         ...   
17800       17806     KANG    30200  Sales - Landscaping       39243   
17801       17807     MILG    30200  Sales - Landscaping       39244   
17802       17808    PETER    30200  Sales - Landscaping       39245   
17803       17809    BICKM    30200  Sales - Landscaping       39253   
17804       17810     MILG    30200  Sales - Landscaping     39244CM   

         Inv_Date  Debit_Amt  Credit_Amt Last_Inv_Date Sales_Rep_ID  \
918    2005-01-19        0.0       180.0    2012-04-23          

In [47]:
#  Determine if customer is a repeat customer. 
#  First sort the dataset by cust ID and Invoice date.  This will group the invoices by Customer to determine which invoices they are repeat customers.
#  Create column 'Repeat_cust' with value ='y' if they are a repeat customer or 'n' if this invoice they are first-time customer.

sales_df = sales_df.sort_values(by=['Cust_ID','Inv_Date'])
gcust = sales_df.groupby("Cust_ID").ngroup()
print(gcust.head(30))
sales_df["Repeat_Cust"] = np.where(gcust.duplicated(), 'y','n')

52339    0
52353    0
4526     0
52399    0
52444    0
52485    0
52523    0
52340    1
52400    1
52445    1
52486    1
52524    1
52559    1
33955    2
41949    2
34025    2
42064    2
34093    2
42177    2
34161    2
49636    2
34230    2
42377    2
34298    2
42511    2
34368    2
42638    2
42697    2
34445    2
34476    2
dtype: int64


In [48]:
# Look to make sure the 'Repeat Customer' column is correct

print(sales_df.sort_values(['Cust_ID','Inv_Date']).head(30))


       Unnamed: 0   Cust_ID  Acct_ID                 Acct_Desc Inv_Num  \
52339       52346       103    30900     Sales - Grounds Maint   14493   
52353       52360       103    30900     Sales - Grounds Maint   14581   
4526         4528       103    30200       Sales - Landscaping   14690   
52399       52406       103    30900     Sales - Grounds Maint   14731   
52444       52451       103    30900     Sales - Grounds Maint   14831   
52485       52492       103    30900     Sales - Grounds Maint   14979   
52523       52530       103    30900     Sales - Grounds Maint   15206   
52340       52347     10975    30900     Sales - Grounds Maint   14494   
52400       52407     10975    30900     Sales - Grounds Maint   14732   
52445       52452     10975    30900     Sales - Grounds Maint   14832   
52486       52493     10975    30900     Sales - Grounds Maint   14980   
52524       52531     10975    30900     Sales - Grounds Maint   15207   
52559       52566     10975    30900  

In [49]:
sales_df.shape  # look at number of columns

(32816, 15)

In [54]:
# If Customer ID is 'WATERO' then change the Acct ID and Acct Desc to 30020 and 'Sales - Turf Repair'

sales_df.loc[sales_df.Cust_ID == 'WATERO',['Acct_ID', 'Acct_Desc']] = '30020', 'Sales - Turf Repair'

#df.loc[df.ID == 103, ['FirstName', 'LastName']] = 'Matt', 'Jones'

In [55]:
sales_df.loc[sales_df['Cust_ID'] == 'WATERO']

,Unnamed: 0,Cust_ID,Acct_ID,Acct_Desc,Inv_Num,Inv_Date,Debit_Amt,Credit_Amt,Last_Inv_Date,Sales_Rep_ID,Job_ID,Inv_Total_Credit,Inv_Total_Debit,Inv_Total_Sale,Repeat_Cust
6007,6009,WATERO,30020,Sales - Turf Repair,19668,2010-04-12,0.0,1328.25,2021-12-29,TOM,"10061-02TB,02-PLANTING,00-REVENUE",23717.90,0.0,23717.90,n
6048,6050,WATERO,30020,Sales - Turf Repair,19709,2010-04-19,0.0,2597.00,2021-12-29,TOM,"10061-02TB,02-PLANTING,00-REVENUE",21297.10,0.0,21297.10,y
6089,6091,WATERO,30020,Sales - Turf Repair,19769,2010-04-26,0.0,1242.00,2021-12-29,TOM,"10061-02TB,02-PLANTING,00-REVENUE",8395.50,0.0,8395.50,y
6171,6173,WATERO,30020,Sales - Turf Repair,19839,2010-04-30,0.0,966.00,2021-12-29,TOM,"10061-02TB,02-PLANTING,00-REVENUE",13437.90,0.0,13437.90,y
6253,6255,WATERO,30020,Sales - Turf Repair,19934,2010-05-10,0.0,3530.00,2021-12-29,TOM,"10061-02TB,02-PLANTING,00-REVENUE",12148.95,0.0,12148.95,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,220,WATERO,30020,Sales - Turf Repair,39210,2021-12-09,0.0,5306.21,2021-12-29,JAMESV,"21001-15JV,15-Turf Repair,00-REVENUE",5306.21,0.0,5306.21,y
222,222,WATERO,30020,Sales - Turf Repair,39230,2021-12-21,0.0,4076.55,2021-12-29,JAMESV,"21001-15JV,15-Turf Repair,00-REVENUE",4076.55,0.0,4076.55,y
223,223,WATERO,30020,Sales - Turf Repair,39231,2021-12-21,0.0,4860.51,2021-12-29,JAMESV,"21001-15JV,15-Turf Repair,00-REVENUE",4860.51,0.0,4860.51,y
224,224,WATERO,30020,Sales - Turf Repair,39232,2021-12-21,0.0,5024.85,2021-12-29,JAMESV,"21001-15JV,15-Turf Repair,00-REVENUE",5024.85,0.0,5024.85,y


In [56]:
# Export Sales file to CSV files
sales_df.to_csv('Sales.csv')